In [10]:
import cv2
import numpy as np
import ipywidgets
from IPython.display import display
import numpy as np
import base64

In [11]:
print(cv2.__version__)

4.8.0-dev


In [12]:
face_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

In [13]:
def get_faces(img, device='cpu'):
    if device == 'cpu':
        face_cascades = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        return face_cascades.detectMultiScale(img, 1.3, 5)
    elif device == 'gpu':
        face_cascades = cv2.cuda_CascadeClassifier.create('./cascades_cuda/haarcascade_frontalface_alt.xml')
        return face_cascades.detectMultiScale(img).download()[0]

In [14]:
def blur_faces(faces, color_img, gray_img):
    for (x, y, w, h) in faces:
        cv2.rectangle(color_img,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,0),thickness=2)
        roi_gray = gray_img[y:y+h, x:x+w]
        roi_color = color_img[y:y+h, x:x+w]
        roi = cv2.GaussianBlur(roi_color, (23, 23), 30)

        color_img[y:y+w, x:x+h] = roi
    return color_img

In [22]:
def face_reduction(color_img, gray_img, device='cpu'):
    faces = get_faces(gray_img, device)
    gray_img = gray_img.download() if device=='gpu' else gray_img
    color_cpu_img = color_img.download() if device=='gpu' else color_img
    blur_face = blur_faces(faces, color_cpu_img, gray_img)
    return blur_face

### Face Redaction with CPU

In [16]:
c_widget = ipywidgets.Image(format='jpeg')
display(c_widget)

cap = cv2.VideoCapture('faceR.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        print(f"Can't receive frames...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = face_reduction(frame, gray, device='cpu')
    c_widget.value = bytes(cv2.imencode('.jpg', frame)[1])

cap.release()
# cv2.destroyAllWindows()

Image(value=b'', format='jpeg')

Can't receive frames...


### Face Reduction with GPU

In [23]:
rimage_widget = ipywidgets.Image(format='jpeg')

display(rimage_widget)

cap = cv2.VideoCapture('faceR.mp4')

while True:
    try:
        ret, frame = cap.read()

        if not ret:
            break

        img_gpu = cv2.cuda_GpuMat()
        img_gpu.upload(frame)
        gray = cv2.cuda.cvtColor(img_gpu, cv2.COLOR_BGR2GRAY)

        frame = face_reduction(img_gpu, gray, device='gpu')
        
        rimage_widget.value = bytes(cv2.imencode('.jpg', frame)[1])
    except Exception as e:
        print(e)
        break

cap.release()
# cv2.destroyAllWindows()


Image(value=b'', format='jpeg')

In [9]:
ipywidgets.Widget.close_all()